In [1]:
import pandas as pd
import numpy as np
import json
import math
from InOutput import InOutput
from CalMeanAllInpatientCycle import CalMeanAllInpatientCycle
from datetime import date, datetime, timedelta

In [2]:
io = InOutput()

In [3]:
io._calculate_group_data()

,x,y
0,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[0, 0, 0, 0, 2013.0, 1382.0, 618.25, 1289.75, ..."


In [ ]:
a = io.handle_instructions(0,0)
a

In [5]:
patient_list = a.iter_patient_list()
cycle_mean_visitor = CalMeanAllInpatientCycle()
for patient in patient_list:
    patient.accept_visitor(cycle_mean_visitor)
    print(cycle_mean_visitor.get_result())

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pand

In [ ]:
a = io.handle_instructions(1,1)
a

In [ ]:
patient_list = a.iter_patient_list()

In [ ]:
#build all patient cycle
inpatient = patient_list[3].get_inpatient()
inpatient

In [ ]:
inpatient['InAge'] = inpatient['InAge'] //5 *5
inpatient

In [ ]:
type(inpatient.loc['13','InDate'])

In [ ]:
def cal_intervel(data):
    data['sn'] = [x for x in range(0,len(data))]
    cur_age = 0
    next_age = 0
    interval_list=[]
    avg_interval=[]
    age_list=[]
    if(len(data) > 1):
        for i in range(0,len(data)-1):
            cur_age = data.loc[data['sn'] == i ,'InAge'].values[0]
            next_age = data.loc[data['sn'] == i+1 ,'InAge'].values[0]
            next_indate= datetime.strptime(data.loc[data['sn'] == i+1, 'InDate'].values[0][0:10], "%Y-%m-%d") 
            cur_indate= datetime.strptime(data.loc[data['sn'] == i ,'InDate'].values[0][0:10], "%Y-%m-%d") 
            days = (next_indate - cur_indate).days
            #days = second.astype('timedelta64[D]') / np.timedelta64(1, 'D')
            interval_list.append(days)
            if(cur_age != next_age):
                avg_interval.append(sum(interval_list)//len(interval_list))
                age_list.append(cur_age)
                interval_list=[]
                interval_list.append(days)        
        if(len(interval_list)!=0):        
            avg_interval.append(sum(interval_list)//len(interval_list))
            age_list.append(next_age)    
        list_index = [x for x in range(0,len(age_list))]
        interval_data = pd.Series(data = avg_interval ,index = list_index)
        age_data = pd.Series(data = age_list ,index = list_index)
        return pd.DataFrame({ 'age':age_data, 'cycle': interval_data})

In [ ]:
a = cal_intervel(inpatient)
a

In [ ]:
def create_complete_data(data):
    default_value = 0
    interval_list = {0:default_value,5:default_value,10:default_value,15:default_value,20:default_value,25:default_value,
                     30:default_value,35:default_value,40:default_value,45:default_value,50:default_value,55:default_value,
                     60:default_value,65:default_value,70:default_value,75:default_value,80:default_value,85:default_value,
                     90:default_value,95: default_value,100:default_value}
    min = data['age'].min()
    max = data['age'].max()
    cur_age_index = -1
    for list_age in interval_list.keys():
        if list_age == min:
            interval_list[list_age] = data.loc[data['age'] == list_age,'interval'].values[0]
            cur_age_index = data.loc[data['age'] == list_age].index.values.astype(int)[0]
        elif list_age == max:
            interval_list[list_age] = data.loc[data['age'] == list_age,'interval'].values[0]
            break;
        elif list_age > min and list_age < max and cur_age_index != -1:
            if list_age < data.loc[data.index == cur_age_index+1,'age'].values[0]:
                interval_list[list_age] = data.loc[data.index == cur_age_index,'interval'].values[0]
            else:
                interval_list[list_age] = data.loc[data.index == cur_age_index+1,'interval'].values[0]
                cur_age_index+=1
    ans = pd.Series(data = list(interval_list.values()) ,index = (interval_list.keys()))
    return pd.DataFrame(ans)


In [ ]:
b = create_complete_data(a)
b = b.reset_index()
b = b.rename(columns = {'index':'age',0:'interval'})
b

In [ ]:
age = b['age'].agg(lambda x: x.tolist())
interval = b['interval'].agg(lambda x: x.tolist())


In [ ]:
result = pd.DataFrame(data = {'x' : [age], 'y' :[interval]})
a = result.to_json(orient='index')
a

In [ ]:
a = None
for i in patient_list:
    if (type(a) == type(None)):
        inpatient = i.get_inpatient()
        if(type(inpatient) != type(None)):
            inpatient['InAge'] = inpatient['InAge'] //5 *5
            a = cal_intervel(inpatient)
            if(type(a) != type(None)):
                a['ID'] = i.get_id()
    else :
        inpatient = i.get_inpatient()
        if(type(inpatient) != type(None)):
            inpatient['InAge'] = inpatient['InAge'] //5 *5
            b = cal_intervel(inpatient)
            if(type(b) != type(None)):
                b['ID'] = i.get_id()
                a = pd.concat([a,b])

In [ ]:
cycle = []
age = []
for i in range (0,101,5):
    age.append(i)
    avg = a.loc[a['age'] == i,'cycle'].mean()
    if(math.isnan(avg)):
        cycle.append(0)
    else:
        cycle.append(avg)
result = pd.DataFrame(data = {'x' : [age], 'y' :[cycle]})
result

None


AttributeError: 'NoneType' object has no attribute 'loc'